webscraping from https://www.imhpa.gob.pa/es/datos-diarios?estacion=1&mes=1&ano=2025 Tocumen enero 2024

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

headers = {"User-Agent": "Mozilla/5.0"}
columnas_finales = [
    "Día",
    "Temperatura Máxima (°C)",
    "Temperatura Mínima (°C)",
    "Temperatura Promedio (°C)",
    "Lluvia Mes Actual (mm)",
    "Lluvia Acumulada(mm)",
    "Lluvia Promedio (mm)",
    "Lluvia Acumulada Promedio Histórico (mm)",
    "Viento Velocidad Máxima (Km/h)",
    "Viento Dirección Máxima (°)",
    "Viento Hora Máxima",
    "Dirección del Viento"
]

data_total = []
for mes in range(1, 13):
    url = f"https://www.imhpa.gob.pa/es/datos-diarios?estacion=6&mes={mes}&ano=2024"
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        tabla = soup.find('table', {'class': 'table'})
        if not tabla:
            print(f"[Aviso] Tabla no encontrada para el mes {mes}")
            continue

        filas = tabla.find_all('tr')
        data = []

        for fila in filas:
            columnas = fila.find_all(['th', 'td'])
            fila_texto = [col.text.strip() for col in columnas]
            if fila_texto:
                data.append(fila_texto)

        df = pd.DataFrame(data)
        df.columns = columnas_finales
        df = df.iloc[2:].reset_index(drop=True)

        # Convertir y limpiar columna Día
        df["Día"] = pd.to_numeric(df["Día"], errors='coerce')
        df = df.dropna(subset=["Día"])
        df = df[df["Día"].between(1, 31)].reset_index(drop=True)

        # Añadir Año y Mes
        df["Año"] = 2024
        df["Mes"] = mes

        # Crear columna Fecha
        df["Día"] = df["Día"].astype(int)
        df_fecha = df.rename(columns={"Año": "year", "Mes": "month", "Día": "day"})
        df["Fecha"] = pd.to_datetime(df_fecha[["year", "month", "day"]], errors='raise')

        columnas_ordenadas = ["Fecha"] + [col for col in df.columns if col != "Fecha"]
        df = df[columnas_ordenadas]

        data_total.append(df)
        time.sleep(1)
        
    except Exception as e:
        print(f"[Error] Fallo en el mes {mes}: {e}")
        continue

if not data_total:
    raise ValueError("No se pudo recolectar ningún mes. Revisa la estructura del sitio o la conexión.")


df_chiriqui = pd.concat(data_total, ignore_index=True)
print("Datos recolectados exitosamente.")
df_chiriqui.to_csv("../datos_crudos/datos_chiriqui_crudos_2024.csv", index=False)


[Error] Fallo en el mes 10: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Datos recolectados exitosamente.


Limpieza de datos

In [3]:
df_chiriqui = df_chiriqui.drop(columns=["Día","Año", "Mes"], errors='ignore')


In [4]:
import numpy as np
df_chiriqui.replace(["", "-", "NA", "N/A", "n/a"], np.nan, inplace=True)


In [5]:
# Porcentaje de valores faltantes
faltantes = df_chiriqui.isna().sum()
print("Valores faltantes por columna:")
print(faltantes[faltantes > 0])
porcentaje_faltantes = (df_chiriqui.isna().sum() / len(df_chiriqui)) * 100
print("\nPorcentaje de valores faltantes:")
print(porcentaje_faltantes[porcentaje_faltantes > 0].round(2))


Valores faltantes por columna:
Temperatura Máxima (°C)           19
Temperatura Mínima (°C)           20
Temperatura Promedio (°C)         21
Lluvia Mes Actual (mm)            17
Lluvia Acumulada(mm)              17
Viento Velocidad Máxima (Km/h)    38
Viento Dirección Máxima (°)       38
Viento Hora Máxima                38
Dirección del Viento              37
dtype: int64

Porcentaje de valores faltantes:
Temperatura Máxima (°C)            5.67
Temperatura Mínima (°C)            5.97
Temperatura Promedio (°C)          6.27
Lluvia Mes Actual (mm)             5.07
Lluvia Acumulada(mm)               5.07
Viento Velocidad Máxima (Km/h)    11.34
Viento Dirección Máxima (°)       11.34
Viento Hora Máxima                11.34
Dirección del Viento              11.04
dtype: float64


In [6]:
import numpy as np

df_chiriqui.replace(["", "-", "NA", "N/A", "n/a"], np.nan, inplace=True)

columnas_numericas = [
    "Temperatura Máxima (°C)",
    "Temperatura Mínima (°C)",
    "Temperatura Promedio (°C)",
    "Lluvia Mes Actual (mm)",
    "Lluvia Acumulada(mm)",
    "Lluvia Promedio (mm)",
    "Lluvia Acumulada Promedio Histórico (mm)",
    "Viento Velocidad Máxima (Km/h)",
    "Viento Dirección Máxima (°)"
]

for col in columnas_numericas:
    df_chiriqui[col] = pd.to_numeric(df_chiriqui[col], errors='coerce')

# Imputar temperaturas con la media
for col in ["Temperatura Máxima (°C)", "Temperatura Mínima (°C)", "Temperatura Promedio (°C)"]:
    df_chiriqui[col] = df_chiriqui[col].fillna(df_chiriqui[col].mean())

# Imputar precipitaciones con 0 = no llovió)
for col in ["Lluvia Mes Actual (mm)", "Lluvia Acumulada(mm)"]:
    df_chiriqui[col] = df_chiriqui[col].fillna(0)

# Imputar viento con la media 
for col in ["Viento Velocidad Máxima (Km/h)", "Viento Dirección Máxima (°)"]:
    df_chiriqui[col] = df_chiriqui[col].fillna(df_chiriqui[col].mean())

# Imputar texto faltante con la moda (valor más frecuente)
for col in ["Dirección del Viento", "Viento Hora Máxima"]:
    if df_chiriqui[col].isna().sum() > 0:
        moda = df_chiriqui[col].mode()
        if not moda.empty:
            df_chiriqui[col] = df_chiriqui[col].fillna(moda[0])

columnas_numericas = df_chiriqui.select_dtypes(include=["number"]).columns
df_chiriqui[columnas_numericas] = df_chiriqui[columnas_numericas].round(1)



Normalizar los datos de temperatura a misma columna

```python

In [7]:
import pandas as pd

columnas_estandar= {
    'Fecha': 'Date',
    'Temperatura Máxima (°C)': 'Tmax',
    'Temperatura Mínima (°C)': 'Tmin',
    'Temperatura Promedio (°C)': 'Tavg',
    'Lluvia Mes Actual (mm)': 'Prcp',
    'Viento Velocidad Máxima (Km/h)': 'Wspd',
    'Dirección del Viento': 'Wdir'
}

# Renombrar columnas y eliminar las demás
df_chiriqui= df_chiriqui.rename(columns=columnas_estandar)  
df_chiriqui = df_chiriqui[list(columnas_estandar.values())] 
df_chiriqui["Estacion"] = "David"
df_chiriqui["Provincia"] = "Chiriquí"


Pasar a CSV

In [8]:
print(df_chiriqui.head())

        Date  Tmax  Tmin  Tavg  Prcp  Wspd         Wdir Estacion Provincia
0 2024-01-01  34.4  23.0  28.7   0.0  43.2        Norte    David  Chiriquí
1 2024-01-02  34.4  23.0  28.7   0.0  43.2        Norte    David  Chiriquí
2 2024-01-03  33.2  22.0  27.6   0.0  28.8          Sur    David  Chiriquí
3 2024-01-04  34.0  22.6  28.3   0.0  30.2  Nor-Noreste    David  Chiriquí
4 2024-01-05  34.2  22.6  28.4   0.0  34.9          Sur    David  Chiriquí


In [ ]:
# Guardar el DataFrame en un archivo CSV
df_chiriqui.to_csv("../datos_limpios/datos_limpios_chiriqui_2024.csv", index=False)